#### ETL Processes

> I would be using this notebook to develop the ETL process for each of the database tables before completing the etl.py file to load the whole datasets.

import libraries

In [1]:
# !pip install psycopg2-binary

In [2]:
import os
import glob
import psycopg2
import pandas as pd 
import sys
import numpy as np
from dotenv import load_dotenv

In [3]:
load_dotenv('../.env')

True

create a database connection

In [4]:
def get_connection(user:str, host:str,database:str, port:str=None, password:str= None) -> psycopg2.extensions.cursor:
    try:
        
            if password is None:
        
                password = os.getenv('PASSWORD')
            
            connection = psycopg2.connect(user = user,
                                          password = password,
                                          host = host,
                                          port = port,
                                          database = database)
            # perform autocommit on queries
            connection.set_session(autocommit=True)
            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print ( connection.get_dsn_parameters(),"\n")

            # Print PostgreSQL version
            cursor.execute("SELECT version();")
            record = cursor.fetchone()
            print("You are connected to - ", record,"\n") 
    except (Exception, psycopg2.Error) as error :
            print ("Error while connecting to PostgreSQL", error)
            sys.exit(1)
    return connection

In [5]:
connection = get_connection(user='postgres', host='127.0.0.1',
              database='moodle')

{'user': 'postgres', 'dbname': 'moodle', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 12.4 (Ubuntu 12.4-1.pgdg18.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit',) 



> define function to execute database queries

In [11]:
def execute_query(query, con, fetch_type:int):
    
    """
    fetch_type:
        1 -> fetchone
        2 -> fetchmany
        3 -> fetchall
    """
    
    try:
        cur = con.cursor() 
        cur.execute(query)
        
        if fetch_type == 1:
            result = cur.fetchone()
        if fetch_type == 2:
            result == cur.fetchmany()
        if fetch_type == 3:
            result = cur.fetchall()
            for row in result:
                print(row)
            
            
    except psycopg2.DatabaseError as e:
        print(f'Error {e}')
        sys.exit(1)
    
    return result

In [12]:
# fetch_tables = ("select * from mdl_course;")
# execute_query(query=fetch_tables, con=connection, fetch_type=3)[0]

#### Tables to be queried in the mooodle database

● mdl_logstore_standard_log -*

● mdl_context *

● mdl_user *

● mdl_course *

● mdl_modules *

● mdl_course_modules *

● mdl_course_modules_completion *

● mdl_grade_items

● mdl_grade_grades *

● mdl_grade_categories

● mdl_grade_items_history

● mdl_grade_grades_history

● mdl_grade_categories_history

● mdl_forum

● mdl_forum_discussions

● mdl_forum_posts

> create a list to store tables names

In [16]:
moodle_tables = ['mdl_logstore_standard_log',
     
'mdl_context',
     
'mdl_user',
     
'mdl_course ',
     
'mdl_modules',
     
'mdl_course_modules ',
     
'mdl_course_modules_completion',
     
'mdl_grade_items',
     
'mdl_grade',

'mdl_grade_categories',

'mdl_grade_items_history',

'mdl_grade_grades_history',

'mdl_grade_categories_history',
     
'mdl_forum',
     
'mdl_forum_discussions',
     
'mdl_forum_posts'
    ]

> write function to parse sql tables to pandas

In [17]:
def get_tables(table_names:list, con) -> dict:
    
    tables = dict()
    for table in table_names:
        
        table_key = table.split('_')[-1]
        
        print("getting table for "+table_key)
        
        tables[table_key] = pd.read_sql_query(sql =f'SELECT * FROM {table}', con=con)
        
    return tables

In [ ]:
tables = get_tables(table_names=moodle_tables, con=connection)

getting table for log


In [ ]:
tables

In [ ]:
tables.keys()

In [ ]:
tables = {'x':x}

In [ ]:
pd.read_sql_query(sql =f'SELECT * FROM mdl_grade_items_history', con=connection)